# 图骗预处理

In [1]:
import os, shutil, platform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import shuffle 

%matplotlib inline 

## 工具函数:判断操作系统种类

In [2]:
def is_Windows_OS():
    if platform.system() == 'Windows':
        return True
    else:
        return False

## 定义的目录结构变量

In [3]:
# 训练集图片从zip包解压之后存放的目录
train_images_folder = 'train'

# 训练集图片目录
train_set_folder = 'train1'
train_set_folder_cat = 'train1/cat/'
train_set_folder_dog = 'train1/dog/'

# 验证集图片目录
validation_set_folder = 'valid'
validation_set_folder_cat = 'valid/cat/'
validation_set_folder_dog = 'valid/dog/'

# 测试集图片从zip包解压缩之后的存放目录
test_set_folder = 'test1'


batch_size = 200
target_size = (250, 250)

## 新建有关目录结构

In [4]:
def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

rmrf_mkdir(train_set_folder)
os.mkdir(train_set_folder_cat)
os.mkdir(train_set_folder_dog)

rmrf_mkdir(validation_set_folder)
os.mkdir(validation_set_folder_cat)
os.mkdir(validation_set_folder_dog)

## 加载训练集目录

In [5]:
train_filenames = os.listdir(train_images_folder)
train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)

In [6]:
print (type(train_cat))
print (len(train_filenames))

<class 'filter'>
25000


In [7]:
print (train_filenames[24999])

cat.8408.jpg


定义一组移动有用的函数：

In [16]:
def move_cat_images():
    timer = 0
    for filename in train_cat:
        if timer < 3000:
            shutil.copyfile('train/'+filename, validation_set_folder_cat + filename)
        else:
            shutil.copyfile('train/'+filename, train_set_folder_cat + filename)
        timer = timer + 1

def move_dog_images():
    timer = 0
    for filename in train_dog:
        if timer < 3000:
            shutil.copyfile('train/'+filename, validation_set_folder_dog + filename)
        else:
            shutil.copyfile('train/'+filename, train_set_folder_dog + filename)
        timer = timer + 1
        
def create_cat_images_symlink():
    timer = 0
    for filename in train_cat:
        if timer <2000:
            os.symlink('train/'+filename, validation_set_folder_cat + filename)
        else:
            os.symlink('train/'+filename, train_set_folder_cat + filename)
        timer = timer + 1

def move_dog_images_symlink():
    timer = 0
    for filename in train_dog:
        if timer < 2000:
            os.symlink('train/'+filename, validation_set_folder_dog + filename)
        else:
            os.symlink('train/'+filename, train_set_folder_dog + filename)
        timer = timer + 1    

In [17]:
def fill_train_set_valid_set_folder_with_images():
    if is_Windows_OS():
        move_cat_images()
        move_dog_images()
    else:
        create_cat_images_symlink()
        move_dog_images_symlink()

In [18]:
fill_train_set_valid_set_folder_with_images()
print ('fill_train_set_valid_set_folder_with_images执行完毕')

fill_train_set_valid_set_folder_with_images执行完毕


# 搭建并且编译模型

In [ ]:
from keras.applications.densenet import DenseNet169
from keras.models import Sequential, Model
from keras.layers import Conv2D, Input
from keras.layers import MaxPooling2D, BatchNormalization
from keras.layers import Flatten, Dropout
from keras.layers import Dense, GlobalAveragePooling2D


# 不包含原有模型的全连接层
dnesenet_model = DenseNet169(include_top=False, 
                             weights='imagenet',
                             input_shape = (250, 250, 3),
                             pooling='avg')

for layer in dnesenet_model.layers:
    layer.trainable = False

x = dnesenet_model.output

# 添加全局平均池化层
# x = GlobalAveragePooling2D()(x)

# Full Connected Layers
# x = Dense(units = 512, activation = 'relu',  name='fc1')(x)

x = Dense(units = 128, activation = 'relu',  name='fc2')(x)

# Dropout Layer
x = Dropout(0.5, name='fc3_dropout')(x)

# Classifier
predictions = Dense(2, activation='softmax', name="predictions")(x)

model = Model(inputs=dnesenet_model.input, outputs=predictions)
# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

## 训练模型

In [15]:

from keras.preprocessing.image import ImageDataGenerator


train_image_gen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_image_gen.flow_from_directory('train1',
                                                 target_size = target_size,
                                                 batch_size = batch_size,
                                                 class_mode = 'binary')
valid_image_gen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

validation_set = valid_image_gen.flow_from_directory('valid',
                                                 target_size = target_size,
                                                 batch_size = batch_size,
                                                 class_mode = 'binary')


model.fit_generator(training_set, steps_per_epoch = 500, epochs = 25, 
                    validation_data=validation_set, validation_steps = 500)

Found 19000 images belonging to 2 classes.
Found 6000 images belonging to 2 classes.
Epoch 1/25


StopIteration: [Errno 2] No such file or directory: 'train1/dog/dog.9915.jpg'